# Example: sphere with PS3 dataset

This notebook shows how to use `ffsas` to invert for the radius distribution of a `Sphere` model from a real SANS dataset called "PS3". We also compare the results from `Irena`, `SasView` and `McSAS`.

In [ ]:
from pathlib import Path

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from scipy import interpolate
import torch

import ffsas
from ffsas.models import Sphere
from ffsas.system import SASGreensSystem

# avoid an OMP error on MacOS (nothing to do with ffsas)
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [ ]:
# reproduce figures in the paper 
reproduce_paper_fig = True
if reproduce_paper_fig:
    # this will trigger an error if latex is not installed
    plt.rcParams.update({
        "text.usetex": True,
        "text.latex.preamble": r'\usepackage{bm,upgreek}',
        "font.family": "sans-serif",
        "font.serif": ["Times"]})
    # figure dir
    paper_fig_dir = Path('../paper_figs')
    Path(paper_fig_dir).mkdir(parents=True, exist_ok=True)

# Read data

Data are stored in the text file `observation.txt`, with the three columns being $q$, mean and standard deviation of the observed intensity.

In [ ]:
# read data
fname = f'observation.txt'
data = np.loadtxt(fname)

# q vector
q = torch.tensor(data[:, 0], dtype=ffsas.torch_dtype)

# intensity mean
mu = torch.tensor(data[:, 1], dtype=ffsas.torch_dtype)

# intensity stddev
sigma = torch.tensor(data[:, 2], dtype=ffsas.torch_dtype)

# McSAS use nm^-1 for q
np.savetxt('observation_McSAS.txt', 
           torch.stack([q * 10, mu, sigma]).t().numpy())

# Inversion

Just a few lines to do inversion:

In [ ]:
# specify radii
r = torch.linspace(400, 800, 1000)

# compute the Green's tensor
truncate = 285  # truncate noise part at high-q
G = Sphere.compute_G_mini_batch([q[:truncate]], {'r': r}, {'drho': 1.})

# build G-system
g_sys = SASGreensSystem(G, Sphere.get_par_keys_G())

# inversion
# do 1000 iterations and save every 100 iterations
results = g_sys.solve_inverse(mu[:truncate], sigma[:truncate], maxiter=1000, save_iter=100, 
                              trust_options={'xtol': 0, 'gtol':0}, verbose=1)

---

# Plot results

First plot `ffsas` results at different iterations.

In [ ]:
plt.rcParams.update({'font.size': 14})
plt.rcParams.update({'legend.fontsize': 14})
plt.rcParams.update({'axes.titlesize': 14})
plt.rcParams.update({'lines.linewidth': 1})

In [ ]:
# volume
v = r ** 3

# colormap
cmap = matplotlib.cm.get_cmap('turbo_r')

# steps to plot (every 100 iters)
n_results = len(results['saved_res'])
plot_steps = range(n_results)

# plot
fig=plt.figure(dpi=200, figsize=(7/1.4, 3.5/1.4))
for j, step in enumerate(plot_steps):
    w = results['saved_res'][step]['w_dict']['r']
    w_hat = w * v / (w * v).sum() * 100  # x100 to percent
    plt.plot(r, w_hat, zorder=-j, c=cmap(step / (n_results - 1)),
             label=r'$w(r)$, iters=%d, wct=%.1f sec' % 
             (results['saved_res'][step]['nit'], 
              results['saved_res'][step]['wct']))
    
plt.xlim(r.min(), r.max())
plt.ylabel(r'Volume weight, $\hat{w}$ (\%)')
plt.xlabel(r'Radius, $r$ (\AA)')
plt.title(r'(a) Convergence of $\hat{w}(r)$ in FFSAS')

# Gaussian approximations of populations
r_ranges=[[420, 500],
          [500, 600],
          [600, 675],
          [675, 800]]
    
for i, (r_min, r_max) in enumerate(r_ranges):
    # find peak
    i_min = torch.argmin(torch.abs(r - r_min))
    i_max = torch.argmin(torch.abs(r - r_max))
    max_loc = torch.argmax(w_hat[i_min:i_max])
    r_top = r[i_min + max_loc]
    
    # find stddev
    area_all = torch.sum(w_hat[i_min:i_max])
    for stddev in range(1, 50):
        area = torch.sum(w_hat[i_min + max_loc - stddev:i_min + max_loc + stddev])
        if area >= area_all * .68:
            break

    # texts
    r_sigma = round(stddev / len(r) * (r.max() - r.min()).item())
    if r_max == 800:
        plt.text(r_top - 1, w_hat[i_min + max_loc] - 2.2, 
                 r'$\mathcal{N}(%d,%d^2)$' % (round(r_top.item()), r_sigma), 
                 ha='left', va='center', fontsize=13, rotation=45)
    else:
        plt.text(r_top - 20, max(w_hat[i_min + max_loc] + 1., .7), 
                 r'$\mathcal{N}(%d,%d^2)$' % (round(r_top.item()), r_sigma), 
                 ha='left', va='bottom', fontsize=13, rotation=45)
plt.ylim(None, 17)

# legend                 
norm = matplotlib.colors.Normalize(vmin=0, vmax=1000)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbaxes = fig.add_axes([0.18, 0.73, 0.45, .025]) 
cb = plt.colorbar(sm, ticks=np.arange(100,1001,100), 
                  boundaries=np.arange(50,1101,100),
                  cax=cbaxes, orientation='horizontal')
cb.ax.tick_params(labelsize=13) 
cb.ax.set_title('Num. iterations', fontsize=13)
cb.ax.tick_params(axis='x', labelrotation = 45)

# save for paper
if reproduce_paper_fig:
    plt.savefig(paper_fig_dir / 'PS3a.pdf', bbox_inches='tight', 
                facecolor='w', pad_inches=.05)
plt.show()

Now we compare the `Irena`, `SasView` and `McSAS` solutions. Because they are very smooth, we compare them to an early `ffsas` result (at 25 iterations).

In [ ]:
nit = 25
results_early = g_sys.solve_inverse(mu[:truncate], sigma[:truncate], maxiter=nit, 
                                    trust_options={'xtol': 0, 'gtol':0}, verbose=1)

In [ ]:
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
style_irena = colors[0], '-.'
style_sasview = colors[1], ':'
style_mcsas = colors[3], '-'
style_ffsas = colors[2], '--'


fig=plt.figure(dpi=200, figsize=(7/1.4, 3.5/1.4))

#### Irena ####
w_hat_irena = np.loadtxt('irena/output_wv(r).txt')
w_hat_irena /= w_hat_irena.sum()
plt.plot(r, w_hat_irena * 100, lw=1.5, c=style_irena[0], ls=style_irena[1], 
         label='Irena')

#### SASView ####
r_mean, PD = 655.31, 0.1113
sigm = r_mean * PD
w_sasview = torch.exp(-.5 * ((r - r_mean) / sigm) ** 2)
w_sasview = w_sasview / w_sasview.sum()
w_hat_sasview = w_sasview * v / torch.sum(w_sasview * v) * 100
plt.plot(r, w_hat_sasview, lw=1.5, c=style_sasview[0], ls=style_sasview[1], 
         label='SasView')

#### MCSAS ####
# read
data = np.loadtxt('mcsas/output_w(r).dat', skiprows=1)
w_mcsas = data[:, 2]
# convert to the same resolution
w_mcsas = interpolate.interp1d(data[:, 0] * 1e10, w_mcsas, fill_value="extrapolate")(r)
w_hat_mcsas = w_mcsas * v.numpy() / (w_mcsas * v.numpy()).sum()
nn = 5
w_hat_mcsas = np.convolve(w_hat_mcsas, np.ones(nn), 'same') / nn
w_hat_mcsas = w_hat_mcsas / w_hat_mcsas.sum()
plt.plot(r, w_hat_mcsas * 100, lw=1, c=style_mcsas[0], ls=style_mcsas[1], 
         label=r'McSAS')

#### FFSAS ####
w_ffsas = results_early['w_dict']['r']
w_hat_ffsas = w_ffsas * v / (w_ffsas * v).sum()
plt.plot(r, w_hat_ffsas * 100, lw=1.5, c=style_ffsas[0], ls=style_ffsas[1], 
         label='FFSAS (%d iters)' % nit)

# figure setting
plt.xlim(400, 800)
plt.ylim(None, .6)
plt.ylabel(r'Volume weight, $\hat{w}$ (\%)')
plt.xlabel(r'Radius, $r$ (\AA)')
plt.legend(prop={'size': 12}, facecolor='whitesmoke')
plt.title(r"(b) $\hat{w}(r)$'s from different methods")

# save for paper
if reproduce_paper_fig:
    plt.savefig(paper_fig_dir / 'PS3b.pdf', bbox_inches='tight', 
                facecolor='w', pad_inches=.05)
plt.show()

Finally, we compare the intensity fit. First, we compute the predicted intensities across the full $q$-range (which just look better than the truncated ones). These intensities are only for plotting; the $\chi^2$ errors are computed based on the actual intensity output from the other codes.

In [ ]:
# full Green's tensor across full q-range
G_full = Sphere.compute_G_mini_batch([q], {'r': r}, {'drho': 1.})
g_sys_full = SASGreensSystem(G_full, Sphere.get_par_keys_G())


#### FFSAS ####
I_ffsas_full = g_sys_full.compute_intensity(w_dict=results_early['w_dict'], 
                                            xi=results['xi'], b=results['b'])
I_ffsas_truncate = I_ffsas_full[:truncate]


#### Irena ####
# I am not extending Irena to full range because I am unsure about its conventions for forward modelling 
I_irena_truncate = np.loadtxt('irena/output_I(q).txt')[:truncate]


#### SasView ####
w_sasview = (w_hat_sasview / r ** 3) / (w_hat_sasview / r ** 3).sum()
scale_sasview, b_sasview = 0.11457, 0.066037
xi_sasview = 1e-4 * scale_sasview / (4 / 3 * np.pi * r ** 3 * w_sasview).sum()
I_sasview_full = g_sys_full.compute_intensity(w_dict={'r': w_sasview}, 
                                              xi=xi_sasview, b=b_sasview)
I_sasview_truncate = np.loadtxt('sasview/output_I(q).txt', skiprows=1)[:, 1]


#### McSAS ####
w_mcsas = (w_hat_mcsas / r ** 3) / (w_hat_mcsas / r ** 3).sum()
scale_mcsas, b_mcsas = 0.109, 0.0831
xi_mcsas = 1e-4 * scale_mcsas / (4 / 3 * np.pi * r ** 3 * w_mcsas).sum()
I_mcsas_full = g_sys_full.compute_intensity(w_dict={'r': w_mcsas}, 
                                            xi=xi_mcsas, b=b_mcsas)
mc_data = np.loadtxt('mcsas/output_I(q).dat', skiprows=1)
q_low = mc_data[:, 0] / 1e10
I_low = mc_data[:, 3]
I_mcsas_truncate = interpolate.interp1d(q_low, I_low, fill_value="extrapolate")(q[:truncate])


#### Monodisperse at 710 A ####
w_delta = results['w_dict']['r'].clone()
w_delta[:] = 0
w_delta[775] = 1  # location of 710 A is 775
# the best-fitting xi and b for monodispersity are found by SasView
scale_delta, b_delta = np.loadtxt('sasview_monodisperse710.txt', skiprows=1)
xi_delta = 1e-4 * scale_delta / (4 / 3 * np.pi * 710 ** 3)
I_delta_full = g_sys_full.compute_intensity(w_dict={'r': w_delta}, 
                                            xi=xi_delta, b=b_delta)
I_delta_truncate = I_delta_full[:truncate]

In [ ]:
matplotlib.rcParams.update({'font.size': 12})
matplotlib.rcParams.update({'legend.fontsize': 12})
matplotlib.rcParams.update({'axes.titlesize': 12})


fig=plt.figure(dpi=200, figsize=(7/1., 2.52/1.))

# data
plt.errorbar(q, mu, yerr=sigma, c='pink', ecolor='skyblue', lw=1, fmt='o',
             markersize=3, label=r'Observation', zorder=-10000)

# Irena
chi2 = np.linalg.norm((mu[:truncate] - I_irena_truncate) / sigma[:truncate]) ** 2
plt.plot(q[:truncate], I_irena_truncate, c=style_irena[0], ls=style_irena[1], lw=2, zorder=0, 
         label=r'Irena: $\chi^2$=%.2f' % (chi2))

# SasView
chi2 = np.linalg.norm((mu[:truncate] - I_sasview_truncate) / sigma[:truncate]) ** 2
plt.plot(q, I_sasview_full, c=style_sasview[0], ls=style_sasview[1], lw=2, zorder=0, 
         label=r'SasView: $\chi^2$=%.2f' % (chi2))

# McSAS
chi2 = np.linalg.norm((mu[:truncate] - I_mcsas_truncate) / sigma[:truncate]) ** 2
plt.plot(q, I_mcsas_full, c=style_mcsas[0], ls=style_mcsas[1], lw=1.5, zorder=0, 
         label=r'McSAS: $\chi^2$=%.2f' % (chi2))

# FFSAS
chi2 = np.linalg.norm((mu[:truncate] - I_ffsas_truncate) / sigma[:truncate]) ** 2
plt.plot(q, I_ffsas_full, c=style_ffsas[0], ls=style_ffsas[1], lw=2, zorder=0, 
         label=f'FFSAS ({nit} iters):')
plt.plot(q, I_ffsas_full, c='whitesmoke', ls=style_ffsas[1], lw=2, zorder=-10000000, 
         label=r'$\chi^2$=%.2f' % (chi2))

# delta
chi2 = np.linalg.norm((mu[:truncate] - I_delta_truncate) / sigma[:truncate]) ** 2
plt.plot(q, I_delta_full, c=colors[4], ls='-', lw=1, zorder=-1000, 
         label=r'Monodisp. $r$=710 \AA:')
plt.plot(q, I_delta_full, c='whitesmoke', ls='-', lw=1, zorder=-100000000, 
         label=r'$\chi^2$=%.2f' % (chi2))

# truncation
plt.axvline(q[truncate], lw=1.5, c='k', ls=':')
plt.text(q[truncate] / 1.05, 10**4.3, 'Truncation', color='k',
         rotation=90, ha='right', va='top', fontsize=11)

plt.xscale('log')
plt.yscale('log')
plt.xlabel('Scattering vector, $q$ (\AA$^{-1}$)')
plt.ylabel('Intensity, $I$ ($\mathrm{cm}^{-1}$)')
handles, labels = plt.gca().get_legend_handles_labels()
handles.insert(0, handles.pop())
labels.insert(0, labels.pop())
plt.legend(handles, labels, loc=[1.02, 0.0], prop={'size': 10}, facecolor='whitesmoke',
           labelspacing=.65)
plt.ylim((10**(-3), 10 ** 4.5))
plt.xlim(q[0] / 1.1, .1)
plt.title(r"(c) Intensity data and $I(q)$'s from different methods")

# save for paper
if reproduce_paper_fig:
    plt.savefig(paper_fig_dir / 'PS3c.pdf', bbox_inches='tight', 
                facecolor='w', pad_inches=.05)
plt.show()

---